In [22]:
import pandas as pd

from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA,ARIMAResults
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from statsmodels.tools.eval_measures import rmse

In [23]:
data = pd.read_csv('co2_mm_mlo.csv')
data.head()

,year,month,decimal_date,average,interpolated
0,1958,3,1958.208,315.71,315.71
1,1958,4,1958.292,317.45,317.45
2,1958,5,1958.375,317.50,317.50
3,1958,6,1958.458,NaN,317.10
4,1958,7,1958.542,315.86,315.86


In [24]:
data.shape

(729, 5)

In [25]:
data.isna().sum()

year            0
month           0
decimal_date    0
average         7
interpolated    0
dtype: int64

In [27]:
data['Date'] = pd.to_datetime({'year':data['year'],'month':data['month'],'day':1})
data.head()

,year,month,decimal_date,average,interpolated,Date
0,1958,3,1958.208,315.71,315.71,1958-03-01
1,1958,4,1958.292,317.45,317.45,1958-04-01
2,1958,5,1958.375,317.50,317.50,1958-05-01
3,1958,6,1958.458,NaN,317.10,1958-06-01
4,1958,7,1958.542,315.86,315.86,1958-07-01


In [28]:
# data = data[['DATETIME','TREATED WATER PRODUCTION IN m3/h','REMARKS']]
# data.head()

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   year          729 non-null    int64         
 1   month         729 non-null    int64         
 2   decimal_date  729 non-null    float64       
 3   average       722 non-null    float64       
 4   interpolated  729 non-null    float64       
 5   Date          729 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 34.3 KB


In [30]:
data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   year          729 non-null    int64         
 1   month         729 non-null    int64         
 2   decimal_date  729 non-null    float64       
 3   average       722 non-null    float64       
 4   interpolated  729 non-null    float64       
 5   Date          729 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 34.3 KB


In [31]:
data.set_index('Date',inplace=True)
data.head()

,year,month,decimal_date,average,interpolated
Date,,,,,
1958-03-01,1958,3,1958.208,315.71,315.71
1958-04-01,1958,4,1958.292,317.45,317.45
1958-05-01,1958,5,1958.375,317.50,317.50
1958-06-01,1958,6,1958.458,NaN,317.10
1958-07-01,1958,7,1958.542,315.86,315.86


In [32]:
data_monthly = data.resample('M')['interpolated'].mean()
data_monthly = data_monthly.reset_index()
data_monthly = data_monthly.set_index('Date')
data_monthly.head()

,interpolated
Date,
1958-03-31,315.71
1958-04-30,317.45
1958-05-31,317.50
1958-06-30,317.10
1958-07-31,315.86


In [33]:
data_monthly.shape

(729, 1)

In [34]:
class TSA:
    def __init__(self,df):
        self.df = df
        
    def adf_test(self,valcol):
        """
        Pass in a time series and an optional title, returns an ADF report
        """
        result = adfuller(self.df[valcol].dropna(),autolag='AIC') # .dropna() handles differenced data
    
        labels = ['ADF test statistic','p-value','# lags used','# observations']
        out = pd.Series(result[0:4],index=labels)

        for key,val in result[4].items():
            out[f'critical value ({key})']=val
        
        print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
        if result[1] <= 0.05:
            print("Strong evidence against the null hypothesis")
            print("Reject the null hypothesis")
            print("Data has no unit root and is stationary")
            state = "Stationary"
        else:
            print("Weak evidence against the null hypothesis")
            print("Fail to reject the null hypothesis")
            print("Data has a unit root and is non-stationary")
            state = "Non-stationary"
        if state == "Stationary":
            return "Yes"
        else:
            return "No"
    def determine_ARIMA_order(self,valcol):
        stepwise_fit = auto_arima(self.df[valcol], start_p=0, start_q=0,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True)           # set to stepwise
        best_order = stepwise_fit.get_params().get('order')
        print('The best order is {}'.format(best_order))
        return best_order
    def fit_model(self,valcol):
#         if len(self.df[valcol]) > 70:
        train = self.df[valcol][:len(self.df[valcol])-8]
        test = self.df[valcol][len(self.df[valcol])-8:len(self.df[valcol])-4]
        val = self.df[valcol][len(self.df[valcol])-4:]
#         else:
#             train = self.df[valcol][:len(self.df[valcol])-8]
#             test = self.df[valcol][len(self.df[valcol])-8:len(self.df[valcol])-4]
#             val = self.df[valcol][len(self.df[valcol])-4:]
        start = len(train)
        end = len(train)+len(test)-1
#             print('train : {}'.format(train))
#             print('test : {}'.format(test))
        print('start : {}'.format(start))
        print('end : {}'.format(end))
        results = ARIMA(train,order=c1.determine_ARIMA_order(valcol)).fit()
        predictions = results.predict(start=start, end=end)
        predictions_val = results.predict(start=end+1, end=len(train)+len(test)+len(val)-1)
        error1 = mean_squared_error(test, predictions)
        error2 = rmse(test, predictions)
        error3 = mean_absolute_percentage_error(test,predictions)
        accuracy = (1-error3)*100
        print(f'MSE Error: {error1:11.10}')
        print(f'RMSE Error: {error2:11.10}')
        print(f'MAPE Error: {error3:11.10}')
        print(f'Accuracy: {accuracy:11.10}')
#         return start, len(test)
        model_results = {"Model":"ARIMA","Stationary": c1.adf_test(valcol),"X_train": str(len(train))+" Weeks", "X_test": str(len(test))+" Weeks", "X_validation": str(len(val))+" Weeks", "ARIMA_order": c1.determine_ARIMA_order(valcol), "MSE": error1,"RMSE": error2,"MAPE":error3, "Accuracy":((1-error3)*100).round(0)}
#         with open('TSA_AQI_model_result_new.json','a') as f:
#             f.write(str(model_results)+',')
#             f.close()
        return predictions_val
        
            
    def full_data_model(self,valcol):
        results = ARIMA(self.df[valcol],order=c1.determine_ARIMA_order(valcol)).fit()
#         if len(self.df[valcol]) > 70:
        fcast = results.predict(len(self.df), len(self.df)+3,typ='levels').round(2)
#         else:
#             fcast = results.predict(len(self.df), len(self.df)+3,typ='levels').round(2)
#         ax = self.df[valcol].plot(legend=True,figsize=(12,6))
#         fcast.plot(legend=True)
        print(fcast)
#         error_rmse = rmse(self.df[valcol],fcast[0:len(self.df)])
#         error_mse = mean_squared_error(self.df[valcol],fcast[0:len(self.df)])
        DF = pd.DataFrame(self.df[valcol])
        DF['Type'] = 'Actual'
#         DF = DF.reset_index()
        DF_fcast = pd.DataFrame(fcast)
        DF_fcast['Type'] = 'Predicted'
        DF_fcast = DF_fcast.rename(columns={'predicted_mean':valcol})
        final_DF = pd.concat([DF,DF_fcast])
        final_DF = final_DF.reset_index()
#         DF_fcast = DF_fcast.rename(columns={'predicted_mean':'Predicted', 'index':'DateTime'})
        DF_val = pd.DataFrame(c1.fit_model(valcol))
        DF_val = DF_val.reset_index()
        DF_val = DF_val.rename(columns={'index':'Date','predicted_mean':'Validation'})
        final_DF = final_DF.rename(columns={'index':'Date'})
        print(DF_val)
        print(final_DF)
#         final_DF =  final_DF.merge(DF_val, on='DateTime',how='outer')
        final_DF =  final_DF.merge(DF_val, on='Date',how='outer')
        final_DF['Date'] = final_DF['Date'].astype('str')
        print(final_DF)
        
#         final_DF.to_json('/Users/syaminiv/Library/CloudStorage/OneDrive-BayesianWaysLLP/Documents/SiAP_ML_Application/Final_Notebooks/June2023/JSON/TSA_withval_Zone_{}.json'.format(item),orient='records')

c1 = TSA(data_monthly)
c1.adf_test("interpolated")
c1.determine_ARIMA_order("interpolated")
c1.fit_model("interpolated")
c1.full_data_model("interpolated")

ADF test statistic        4.994847
p-value                   1.000000
# lags used              20.000000
# observations          708.000000
critical value (1%)      -3.439620
critical value (5%)      -2.865631
critical value (10%)     -2.568948
Weak evidence against the null hypothesis
Fail to reject the null hypothesis
Data has a unit root and is non-stationary
The best order is (2, 1, 1)
start : 721
end : 724
The best order is (2, 1, 1)


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


MSE Error: 2.981316143
RMSE Error: 1.726648819
MAPE Error: 0.003836873286
Accuracy: 99.61631267
ADF test statistic        4.994847
p-value                   1.000000
# lags used              20.000000
# observations          708.000000
critical value (1%)      -3.439620
critical value (5%)      -2.865631
critical value (10%)     -2.568948
Weak evidence against the null hypothesis
Fail to reject the null hypothesis
Data has a unit root and is non-stationary
The best order is (2, 1, 1)
The best order is (2, 1, 1)


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


2018-12-31    410.00
2019-01-31    411.35
2019-02-28    411.79
2019-03-31    411.34
Freq: M, Name: predicted_mean, dtype: float64
start : 721
end : 724
The best order is (2, 1, 1)


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


MSE Error: 2.981316143
RMSE Error: 1.726648819
MAPE Error: 0.003836873286
Accuracy: 99.61631267
ADF test statistic        4.994847
p-value                   1.000000
# lags used              20.000000
# observations          708.000000
critical value (1%)      -3.439620
critical value (5%)      -2.865631
critical value (10%)     -2.568948
Weak evidence against the null hypothesis
Fail to reject the null hypothesis
Data has a unit root and is non-stationary
The best order is (2, 1, 1)
        Date  Validation
0 2018-08-31  406.468010
1 2018-09-30  406.036312
2 2018-10-31  406.076947
3 2018-11-30  406.496889
          Date  interpolated       Type
0   1958-03-31        315.71     Actual
1   1958-04-30        317.45     Actual
2   1958-05-31        317.50     Actual
3   1958-06-30        317.10     Actual
4   1958-07-31        315.86     Actual
..         ...           ...        ...
728 2018-11-30        408.02     Actual
729 2018-12-31        410.00  Predicted
730 2019-01-31        411.

### TSA class with seasonality

In [35]:
class TSA_seasonal:
    def __init__(self, df):
        self.df = df

    def adf_test(self, valcol):
        """
        Pass in a time series and an optional title, returns an ADF report
        """
        result = adfuller(self.df[valcol].dropna(), autolag='AIC')  # .dropna() handles differenced data

        labels = ['ADF test statistic', 'p-value', '# lags used', '# observations']
        out = pd.Series(result[0:4], index=labels)

        for key, val in result[4].items():
            out[f'critical value ({key})'] = val

        if result[1] <= 0.05:
            state = "Stationary"
        else:
            state = "Non-stationary"
        return state

    def determine_ARIMA_order(self, valcol):

        stepwise_fit = auto_arima(self.df[valcol], seasonal=True, m=12,
                                  start_p=0, start_q=0,
                                  error_action='ignore',  # we don't want to know if an order does not work
                                  suppress_warnings=True,  # we don't want convergence warnings
                                  stepwise=True)  # set to stepwise
        best_order = stepwise_fit.get_params().get('order')
        print('The best order is {}'.format(best_order))
        return best_order

    def fit_model(self, valcol):
        train = self.df[valcol][:len(self.df[valcol]) - 8]
        test = self.df[valcol][len(self.df[valcol]) - 8:len(self.df[valcol]) - 4]
        val = self.df[valcol][len(self.df[valcol]) - 4:]

        results = ARIMA(train, order=self.determine_ARIMA_order(valcol)).fit()
        predictions = results.predict(start=len(train), end=len(train) + len(test) - 1)
        predictions_val = results.predict(start=len(train) + len(test), end=len(train) + len(test) + len(val) - 1)
        error1 = mean_squared_error(test, predictions)
        error2 = rmse(test, predictions)
        error3 = mean_absolute_percentage_error(test,predictions)
        accuracy = (1-error3)*100
        print(f'MSE Error: {error1:11.10}')
        print(f'RMSE Error: {error2:11.10}')
        print(f'MAPE Error: {error3:11.10}')
        print(f'Accuracy: {accuracy:11.10}')
        return predictions_val

    def full_data_model(self, valcol):
        results = ARIMA(self.df[valcol], order=self.determine_ARIMA_order(valcol)).fit()
        fcast = results.forecast(steps=4).round(2)  # Forecast 4 steps ahead
#         print(fcast)
#         fcast_index = pd.date_range(start=self.df.index[-1], periods=4 + 1, freq='M')[1:]  # Assuming monthly data
        DF_fcast = pd.DataFrame({valcol: fcast})
        DF_fcast['Type'] = 'Predicted'
        print(DF_fcast)
        DF = pd.DataFrame(self.df[valcol])
        DF['Type'] = 'Actual'
        # Concatenate original data and forecast data
        combined_DF = pd.concat([DF, DF_fcast])
        combined_DF = combined_DF.reset_index().rename(columns={'index':'Date'})
        print(combined_DF)
        DF_val = pd.DataFrame(c2.fit_model(valcol))
        DF_val = DF_val.reset_index()
        DF_val = DF_val.rename(columns={'index':'Date','predicted_mean':'Validation'})
#         combined_DF = combined_DF.rename(columns={'index':'Date'})
        print(DF_val)
        print(combined_DF)
#         final_DF =  final_DF.merge(DF_val, on='DateTime',how='outer')
        final_DF =  combined_DF.merge(DF_val, on='Date',how='outer')
        final_DF['Date'] = final_DF['Date'].astype('str')
#         print(combined_df)
        return final_DF
    
    
c2 = TSA_seasonal(data_monthly)
c2.adf_test("interpolated")
c2.determine_ARIMA_order("interpolated")
c2.fit_model("interpolated")
c2.full_data_model("interpolated")

The best order is (2, 1, 1)
The best order is (2, 1, 1)


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


MSE Error: 2.981316143
RMSE Error: 1.726648819
MAPE Error: 0.003836873286
Accuracy: 99.61631267
The best order is (2, 1, 1)


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


            interpolated       Type
2018-12-31        410.00  Predicted
2019-01-31        411.35  Predicted
2019-02-28        411.79  Predicted
2019-03-31        411.34  Predicted
          Date  interpolated       Type
0   1958-03-31        315.71     Actual
1   1958-04-30        317.45     Actual
2   1958-05-31        317.50     Actual
3   1958-06-30        317.10     Actual
4   1958-07-31        315.86     Actual
..         ...           ...        ...
728 2018-11-30        408.02     Actual
729 2018-12-31        410.00  Predicted
730 2019-01-31        411.35  Predicted
731 2019-02-28        411.79  Predicted
732 2019-03-31        411.34  Predicted

[733 rows x 3 columns]
The best order is (2, 1, 1)


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


MSE Error: 2.981316143
RMSE Error: 1.726648819
MAPE Error: 0.003836873286
Accuracy: 99.61631267
        Date  Validation
0 2018-08-31  406.468010
1 2018-09-30  406.036312
2 2018-10-31  406.076947
3 2018-11-30  406.496889
          Date  interpolated       Type
0   1958-03-31        315.71     Actual
1   1958-04-30        317.45     Actual
2   1958-05-31        317.50     Actual
3   1958-06-30        317.10     Actual
4   1958-07-31        315.86     Actual
..         ...           ...        ...
728 2018-11-30        408.02     Actual
729 2018-12-31        410.00  Predicted
730 2019-01-31        411.35  Predicted
731 2019-02-28        411.79  Predicted
732 2019-03-31        411.34  Predicted

[733 rows x 3 columns]


,Date,interpolated,Type,Validation
0,1958-03-31,315.71,Actual,NaN
1,1958-04-30,317.45,Actual,NaN
2,1958-05-31,317.50,Actual,NaN
3,1958-06-30,317.10,Actual,NaN
4,1958-07-31,315.86,Actual,NaN
...,...,...,...,...
728,2018-11-30,408.02,Actual,406.496889
729,2018-12-31,410.00,Predicted,NaN
730,2019-01-31,411.35,Predicted,NaN
731,2019-02-28,411.79,Predicted,NaN
